In [44]:
from sentence_transformers import SentenceTransformer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama
import faiss
import pandas as pd
import numpy as np
import pickle
import time
from tqdm import tqdm

In [4]:
# Load the FAISS index
index = faiss.read_index("database/pdf_sections_index.faiss")

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
def search_faiss(query, k=3):
    query_vector = model.encode([query])[0].astype('float32')
    query_vector = np.expand_dims(query_vector, axis=0)
    distances, indices = index.search(query_vector, k)
    
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        results.append({
            'distance': dist,
            'content': sections_data[idx]['content'],
            'metadata': sections_data[idx]['metadata']
        })
    
    return results

In [33]:
# Create a prompt template
prompt_template = """
You are an AI assistant specialized in dietary guidelines. Use the following pieces of context to answer the question at the end under 30 words. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Answer:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

llm = Ollama(
    model="llama3"
)

In [34]:
# Create the chain
chain = LLMChain(llm=llm, prompt=prompt)

def answer_question(query):
    # Search for relevant context
    search_results = search_faiss(query)
    
    # Combine the content from the search results
    context = "\n\n".join([result['content'] for result in search_results])

    # Run the chain
    response = chain.run(context=context, question=query)
    
    return response

In [32]:
qa_pairs = pd.read_csv('data/QA_pairs.csv')

In [45]:
time_list=[]
response_list=[]
for i in tqdm(range(len(qa_pairs))):
    query = qa_pairs['Questions'].values[i]
    start = time.time()
    response = answer_question(query)
    end = time.time()   
    time_list.append(end-start)
    response_list.append(response)

100%|███████████████████████████████████████████| 10/10 [00:46<00:00,  4.68s/it]


In [27]:
# Example usage
query = "What are the most common mental disorders mentioned in the handbook?"

In [16]:
# Example usage
query = "What are the most common mental disorders mentioned in the handbook?"

In [17]:
results = search_faiss(query)

In [13]:
with open('database/pdf_sections_data.pkl', 'rb') as f:
        sections_data = pickle.load(f)

In [21]:
import pandas as pd
qa_pairs = pd.read_csv('data/QA_pairs.csv')

In [ ]:
for 